In [1]:
#Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 101)

In [2]:
file_path = './employee.csv'
data = pd.read_csv(file_path)
data.head()

,id,timestamp,country,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,1,12/11/2018 10:52:26,Slovenia,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,2,1/5/2017 16:57:50,United States,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,3,12/18/2017 8:13:15,Sweden,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,4,12/27/2018 4:56:52,United States,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,5,12/11/2018 14:07:58,United States,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [3]:
data = data.drop(columns=['id', 'timestamp','country'])

In [4]:
data.columns

Index(['employment_status', 'job_title', 'job_years', 'is_manager',
       'hours_per_week', 'telecommute_days_per_week', 'education',
       'is_education_computer_related', 'certifications', 'salary'],
      dtype='object')

In [5]:
data.shape

(4277, 10)

In [6]:
# replace NANs in hours_per_week with median value of the column
data.loc[data['hours_per_week'].isna(), 'hours_per_week'] = data['hours_per_week'].median()
data.loc[data['telecommute_days_per_week'].isna(), 'telecommute_days_per_week'] = data['telecommute_days_per_week'].median()
     
#Handling null values in categorical columns
data = data.dropna()

In [7]:
data.shape

(4261, 10)

In [31]:
#Encoding binary variables
binary_cols = ['is_manager', 'certifications']
for c in binary_cols:
    data[c] = data[c].replace(to_replace=['Yes'], value=1)
    data[c] = data[c].replace(to_replace=['No'], value=0)
# select categorical features
cat_cols = [c for c in data.columns if data[c].dtype == 'object'
            and c not in ['is_manager', 'certifications']]
cat_data = data[cat_cols]
final_data = pd.get_dummies(data, columns=cat_cols, drop_first= True,dtype=int)
final_data

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,salary,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
0,4.783930,1,40.0,0.0,0,7187.743094,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
1,5.000000,0,40.0,5.0,0,10000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,1.000000,0,40.0,0.0,1,7000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,1.000000,0,40.0,2.0,0,8333.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,3.000000,0,40.0,2.0,1,7137.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,2.000000,1,40.0,0.0,0,4917.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4273,4.140793,1,40.0,0.0,0,7033.845701,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4274,5.000000,0,40.0,0.0,0,7166.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4275,4.000000,1,40.0,2.0,0,9583.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [39]:
y = final_data['salary']
X = final_data.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training Set Dimensions:", y_train.shape)
print("Validation Set Dimensions:", y_test.shape)

Training Set Dimensions: (3408,)
Validation Set Dimensions: (853,)


In [33]:
num_cols = ['job_years','hours_per_week','telecommute_days_per_week']
# Apply standard scaling on numeric data
scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

In [34]:
X_train

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
2267,0.266336,0,-0.486220,0.590377,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3197,1.780080,0,-1.619921,-0.014192,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2958,-0.238245,0,-0.486220,-0.618761,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4226,-1.247407,1,-0.486220,-0.014192,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3610,0.266336,0,-0.486220,-0.618761,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964,0.266336,0,-0.486220,-0.014192,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3800,0.318603,0,0.154576,2.404084,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3145,2.789243,0,-0.486220,-0.014192,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3867,0.266336,1,-0.486220,0.590377,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [35]:
#Fitting a Linear Regression Model
reg=LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [40]:
# Predict on the test data
y_pred = reg.predict(X_test)
#Evaluate the model on test data
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_pred, y_test)/np.mean(y_test)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

Mean Absolute Error: 9121.643759928931
Mean Squared Error: 12096.293602220683
